In [1]:
import warnings
warnings.filterwarnings("ignore")
import wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
import kc_wrangle as w
import wrangle as ww


# Acquire

In [2]:
df = pd.read_csv("merged_data - merged_data.csv")

In [3]:
df, model_df = w.get_prep_aa(df)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144627 entries, 0 to 197203
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   has_name          144627 non-null  int64 
 1   outcome           144627 non-null  object
 2   dob               144627 non-null  object
 3   species           144627 non-null  object
 4   intake_type       144627 non-null  object
 5   intake_condition  144627 non-null  object
 6   intake_date       144627 non-null  object
 7   outcome_date      144627 non-null  object
 8   intake_age        144627 non-null  object
 9   outcome_age       144627 non-null  object
 10  tenure_days       144627 non-null  object
 11  intake_sex        144627 non-null  object
 12  breed             144627 non-null  object
 13  mix_breeds        144627 non-null  int64 
 14  two_breeds        144627 non-null  int64 
 15  pure_breed        144627 non-null  int64 
 16  primary_color     144627 non-null  obj

* lowercase variables
* intake age/ outcome age leave just days
* name column made need to be dropped from too many nulls 
* animal id column change to id 


In [ ]:
df.head()

# Prepare

In [ ]:
#find numeric columns
num_col = df.select_dtypes(include='number').columns
num_col

In [ ]:
#get string columns
str_col = ['outcome', 'intake_type', 'intake_condition',
       'species', 'found_location', 'intake_sex', 'breed', 'color']

In [ ]:
df.intake_sex.value_counts()

In [ ]:
df.intake_condition.value_counts()

rename intake_conditions: Normal, Injured, Immediate attn needed, mental, delayed/terminal

In [ ]:
df.head()

In [ ]:
# lowercase everything
df = df.apply(lambda x: x.astype(str).str.lower())

# top ten colors and other for erybody else, or for simplify all colors

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion